<h1 style="font-weight:bold">Qual o Efeito da Legalização da Maconha na Taxa de Criminalidade ao Longo do Tempo?</h1>

<hr/>
<h2>Parte 2 - Análise Exploratória com Geolocalização e Modelagem Estatística</h2>
<hr/>

<h2>1) INTRODUÇÃO</2>

Para começar a parte 2 precisaremos dos 4 datasets gerados na primeira parte deste projeto:
* df_crimes.csv
* df_escolas.csv
* df_lojas.csv
* df_prisoes.csv

<h3>1.1) O que faremos na Parte 2:</h3>

* Análise exploratória dos dados.
* Construção de gráficos com Geolocalização, mostrando a localidade exata de ocorrência de cada evento.
* Transformação e segmentação dos dados por período de tempo.
* Cálculo de distância entre os eventos com base na geolocalização usando o pacote geopy.
* Resumo estatístico das variáveis.
* Análise e interpretação das distribuição das variáveis.
* Teste de normalidade para verificar se as amostras de dados foram extraídas de uma população que segue uma distribuição normal.
* Análise e interpretação do valor-p.
* Conclusões da análise exploratória.

<h3>1.2) Carregando os Pacotes Usados Neste Jupyter Notebook</h3>

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Imports para manipulação e visualização de dados
import scipy
import pandas as pd
from scipy import stats

#Imports para realizar os gráficos
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

# Imports para formatação dos gráficos
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
from matplotlib.pylab import rcParams 
rcParams['figure.figsize'] = 15,7
plt.style.use('fivethirtyeight')
%matplotlib inline

<hr/>
<h2>2) ANÁLISE EXPLORATÓRIA</h2>

Na Parte 1 do projeto trabalhamos na extração dos dados. Fizemos uma série de limpezas e transformações e deixamos os dados organizados para a etapa seguinte, na qual trabalharemos agora: análise exploratória e modelagem estatística. Nosso objetivo é compreender os dados, identificar e corrigir eventuais problemas que eles possam apresentar.

<h3>2.1) Número de Prisões Associadas à Maconha </h3>

Nossa primeira tarefa será analisar as prisões associadas à maconha, o objetivo principal do nosso estudo.

In [2]:
# Carregando os dados sobre prisões
df_prisoes = pd.read_csv('dados/df_prisoes.csv')
# Shape
df_prisoes.shape

(20673, 21)

In [3]:
df_prisoes.head()

,Unnamed: 0,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,...,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,latitude,longitude,marijuana_related
0,340,5568617,2019-03-09,2015.0,6,Hollywood,646,29,M,O,...,Narcotic Drug Laws,F,11359HS,possession marijuana for sale,CHEROKEE,HOLLYWOOD,"['34.1016', '-118.335']",34.1016,-118.3350,possession marijuana for sale
1,342,5568629,2019-03-09,2015.0,6,Hollywood,646,25,M,B,...,Narcotic Drug Laws,F,11359HS,possession marijuana for sale,CHEROKEE,HOLLYWOOD,"['34.1016', '-118.335']",34.1016,-118.3350,possession marijuana for sale
2,983,5607178,2019-04-21,2115.0,7,Wilshire,702,42,M,B,...,Narcotic Drug Laws,F,11359(D)HS,poss/sale marij ovr 21 employ per 20/belw,MELROSE AV,FAIRFAX AV,"['34.0838', '-118.3614']",34.0838,-118.3614,poss/sale marij ovr 21 employ per 20/belw
3,1249,200608837,2020-03-17,2200.0,6,Hollywood,646,21,F,B,...,Narcotic Drug Laws,I,11362.3A1HS,smoke/ingest marijuana in public place,HOLLYWOOD,VINE,"['34.1016', '-118.3267']",34.1016,-118.3267,smoke/ingest marijuana in public place
4,1255,200608903,2020-03-19,2320.0,6,Hollywood,645,25,M,W,...,Narcotic Drug Laws,I,11362.3A1HS,smoke/ingest marijuana in public place,HOLLYWOOD,HIGHLAND,"['34.1016', '-118.3387']",34.1016,-118.3387,smoke/ingest marijuana in public place


In [4]:
# Resumo estatístico
df_prisoes.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,20673.0,5.421069e+05,3.585493e+05,340.0000,1.968940e+05,5.307320e+05,8.068310e+05,1.349967e+06
Report ID,20673.0,5.327595e+07,5.941124e+07,2586.0000,3.119415e+06,4.815994e+06,1.103068e+08,2.019007e+08
Time,20670.0,1.523880e+03,5.301552e+02,1.0000,1.245000e+03,1.615000e+03,1.905000e+03,2.358000e+03
Area ID,20673.0,1.098094e+01,6.080943e+00,1.0000,6.000000e+00,1.200000e+01,1.600000e+01,2.100000e+01
Reporting District,20673.0,1.146309e+03,6.065064e+02,100.0000,6.450000e+02,1.239000e+03,1.656000e+03,2.197000e+03
Age,20673.0,2.903749e+01,1.124652e+01,11.0000,2.000000e+01,2.600000e+01,3.500000e+01,7.900000e+01
Charge Group Code,20673.0,1.600000e+01,0.000000e+00,16.0000,1.600000e+01,1.600000e+01,1.600000e+01,1.600000e+01
latitude,20673.0,3.405314e+01,7.200008e-01,0.0000,3.399330e+01,3.404820e+01,3.416760e+01,3.454770e+01
longitude,20673.0,-1.183042e+02,2.471156e+00,-118.6647,-1.184345e+02,-1.183306e+02,-1.182762e+02,0.000000e+00


Vejamos qual é o range de data das prisões.

In [5]:
print('As prisões ocorreram entre {} e {}'.format(df_prisoes['Arrest Date'].min(), df_prisoes['Arrest Date'].max()))

As prisões ocorreram entre 2010-01-01 e 2020-05-08


Não precisamos analisar as prisões em todas as datas. Vamos considerar somente as prisões entre 2014 e 2019, pois esse período compreende o antes e depois da data de legalização da maconha. A maconha foi legalizada em 2016 e liberação entrou em vigor em 2017. Assim teremos dados de 3 anos antes da liberação e 3 anos depois. Isso deve ser suficiente para a análise.

In [6]:
# Filtra o dataframe mantendo apenas os registros entre 2014 e 2019.
df_prisoes = df_prisoes[(df_prisoes['Arrest Date'] >= '2014-01-01') & (df_prisoes['Arrest Date'] <= '2019-12-31')]

# Shape
df_prisoes.shape

(6226, 21)

Agora vamos salvar o resultado em um arquivo csv.

In [7]:
# Salva o arquivo em disco (agora sem o índice)
df_prisoes.to_csv('dados/df_prisoes_parte2.csv', index = False)

Criaremos um gráfico de prisões considerando Geolocalização, ou seja, dados de latitude e longitude. Isso é mais fácil do que parece, desde que os dados estejam disponíveis.

In [8]:
df_prisoes.head()

,Unnamed: 0,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,...,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,latitude,longitude,marijuana_related
0,340,5568617,2019-03-09,2015.0,6,Hollywood,646,29,M,O,...,Narcotic Drug Laws,F,11359HS,possession marijuana for sale,CHEROKEE,HOLLYWOOD,"['34.1016', '-118.335']",34.1016,-118.3350,possession marijuana for sale
1,342,5568629,2019-03-09,2015.0,6,Hollywood,646,25,M,B,...,Narcotic Drug Laws,F,11359HS,possession marijuana for sale,CHEROKEE,HOLLYWOOD,"['34.1016', '-118.335']",34.1016,-118.3350,possession marijuana for sale
2,983,5607178,2019-04-21,2115.0,7,Wilshire,702,42,M,B,...,Narcotic Drug Laws,F,11359(D)HS,poss/sale marij ovr 21 employ per 20/belw,MELROSE AV,FAIRFAX AV,"['34.0838', '-118.3614']",34.0838,-118.3614,poss/sale marij ovr 21 employ per 20/belw
6,2410,5616516,2019-05-01,1045.0,14,Pacific,1494,26,M,B,...,Narcotic Drug Laws,F,11360A3DHS,sale/trans >28.5g marijuana or >4g,500 WORLD WY,NaN,"['33.9428', '-118.4054']",33.9428,-118.4054,sale/trans >28.5g marijuana or >4g
8,2913,5616188,2019-05-01,5.0,15,N Hollywood,1543,27,M,B,...,Narcotic Drug Laws,F,11360A3DHS,sale/trans >28.5g marijuana or >4g,LAUREL CANYON BL,WEDDINGTON,"['34.1667', '-118.3965']",34.1667,-118.3965,sale/trans >28.5g marijuana or >4g


In [9]:
df_prisoes['Arrest Date'] = pd.to_datetime(df_prisoes['Arrest Date'])
df_prisoes['Year'] = df_prisoes['Arrest Date'].dt.year.astype(str)

In [10]:
fig = px.scatter(df_prisoes, x='latitude', y='longitude', color='Year')
fig.update_layout(
    title_text="<span>Total de prisões por ano</span>",
    autosize=False,
    width=1300,
    height=700,
)

fig.show()

In [11]:
df_prisoes.groupby('Year')['Year'].count()

Year
2014    1705
2015    1535
2016    1386
2017     528
2018     564
2019     508
Name: Year, dtype: int64

Os números (e os gráficos) mostram que o total de prisões diminuiu de 2014 a 2019. Não podemos afirmar categoricamente que foi devido à legalização da maconha, mas considerando que este foi o grande evento ocorrido no período (aprovado em 2016 e em vigor a partir de 01/01/2017), já podemos perceber uma tendência, afinal, o uso recreativo da maconha deixou de ser ocorrência de polícia.

Vejamos o que aconteceu nas áreas próximas aos pontos de venda legalizada de maconha.

<h3>2.2) Prisões Associadas à Maconha e Que Ocorreram Próximo a Pontos de Venda Legalizada de Maconha</3>

In [12]:
# Carrega o dataset
df_lojas = pd.read_csv('dados/df_lojas.csv')

# Shape
df_lojas.shape

(385, 10)

In [13]:
# Dados
df_lojas.head(3)

,Unnamed: 0,name,is_closed,url,rating,price,review_count,latitude,longitude,location
0,0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,$$,364,34.08236,-118.271930,"(34.08236, -118.27193)"
1,1,Herbarium,False,https://www.yelp.com/biz/herbarium-los-angeles...,4.5,$$,417,34.08854,-118.344699,"(34.08854, -118.344699)"
2,2,MedMen- Los Angeles - DTLA,False,https://www.yelp.com/biz/medmen-los-angeles-dt...,4.0,$$,451,34.04460,-118.254440,"(34.0446, -118.25444)"


Uma das colunas indica se o ponto de venda está fechado ou não (is_closed). Pontos de venda fechados não trazem informação útil. Vamos checar.

In [14]:
# Verificando se há pontos de venda com status "fechado"
df_lojas['is_closed'].value_counts()

False    385
Name: is_closed, dtype: int64

Não há nenhum ponto de venda com status de fechado, portanto não teremos que nos preocupar com esse fator.

A variável **is_closed** já não fornece nenhuma informação valiosa e a variável **location** é redundante devido às variáveis **latitude** e **longitude**. Além disso, a coluna **url** não é necessária para fins de análise exploratória de dados, portanto as três variáveis serão descartadas. As variáveis rating, price e review_count também não são úteis para esta análise.

De fato, precisamos somente do nome, latitude e longitude.

In [15]:
# Filtrando o dataframe e mantendo somente as colunas necessárias para análise
df_lojas = df_lojas[['name', 'latitude', 'longitude']]

# Dados
df_lojas.head(5)

,name,latitude,longitude
0,California Caregivers Alliance,34.082360,-118.271930
1,Herbarium,34.088540,-118.344699
2,MedMen- Los Angeles - DTLA,34.044600,-118.254440
3,Kushfly,34.127617,-118.346710
4,The Higher Path,34.149266,-118.439963


In [16]:
# Resumo estatístico
df_lojas.describe()

,latitude,longitude
count,385.000000,385.000000
mean,34.080912,-118.115188
std,0.412341,2.801934
min,33.766730,-118.848091
25%,34.007280,-118.378990
50%,34.048006,-118.322359
75%,34.094340,-118.253020
max,40.752290,-73.990005


In [17]:
# Salva o arquivo em disco (agora sem o índice)
df_lojas.to_csv('dados/df_lojas_parte2.csv', index = False)

In [41]:
df_lojas = df_lojas.loc[df_lojas['longitude'] < -100]

Vamos colocar no gráfico a Geolocalização das prisões e das lojas e avaliamos se há algum padrão a considerar.

In [70]:
# Função para o plot das prisões e das lojas, considerando Geolocalização pela latitude e longitude
def grafico_prisoes_lojas(year):
    
    df_lojas_temp = df_lojas[['latitude', 'longitude']]
    df_lojas_temp['Type'] = 'store'

    df_prisoes_temp = df_prisoes.loc[df_prisoes['Year'] == year, ['latitude', 'longitude']]
    df_prisoes_temp['Type'] = 'prison'

    df_lojas_prisoes = pd.concat([df_prisoes_temp, df_lojas_temp], ignore_index=True)


    fig = px.scatter(df_lojas_prisoes, x='latitude', y='longitude', color='Type', color_discrete_map={'store': '#00bf33', 'prison': '#F00'})


    fig.update_layout(
        title_text="<span>Total de prisões e lojas no ano de {}</span>".format(year),
        autosize=False,
        width=1200,
        height=600,
    )

    fig.show()  



In [71]:
df_prisoes.loc[df_prisoes['Year'] == '2014']

,Unnamed: 0,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,...,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,latitude,longitude,marijuana_related,Year
283,44052,4076177,2014-08-27,730.0,15,N Hollywood,1513,19,F,H,...,F,11360(A)HS,transport/sell/furnish/etc marijuana,6800 WHITSETT AV,NaN,"['34.194', '-118.4052']",34.1940,-118.4052,transport/sell/furnish/etc marijuana,2014
284,44056,4128545,2014-10-16,2125.0,10,West Valley,1024,19,M,H,...,F,11359HS,possession marijuana for sale,RESEDA AV,VANOWEN ST,"['34.1939', '-118.536']",34.1939,-118.5360,possession marijuana for sale,2014
299,45273,141005328,2014-02-02,940.0,10,West Valley,1079,40,F,W,...,M,11357BHS,possess 28.5 grams or less of marijuana**,15400 VENTURA BL,NaN,"['34.1599', '-118.4766']",34.1599,-118.4766,possess 28.5 grams or less of marijuana**,2014
642,96985,141013196,2014-08-04,2030.0,10,West Valley,1061,21,M,H,...,I,11357(B)HS,possess 28.5 grams or less of marijuana,19800 VENTURA BL,NaN,"['34.1725', '-118.5638']",34.1725,-118.5638,possess 28.5 grams or less of marijuana,2014
667,99556,3829907,2014-01-10,1950.0,5,Harbor,513,31,M,W,...,F,11359HS,possession marijuana for sale,1200 W 255TH ST,NaN,"['33.7928', '-118.2975']",33.7928,-118.2975,possession marijuana for sale,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17683,1008398,4196674,2014-12-31,2325.0,14,Pacific,1441,21,M,W,...,F,11359HS,possession marijuana for sale,2600 GRAND CANAL,NaN,"['33.9825', '-118.4667']",33.9825,-118.4667,possession marijuana for sale,2014
17684,1008421,4196499,2014-12-31,2200.0,6,Hollywood,643,22,M,B,...,F,11359HS,possession marijuana for sale,LA BREA,HOLLYWOOD,"['34.1016', '-118.3452']",34.1016,-118.3452,possession marijuana for sale,2014
17690,1008867,4195937,2014-12-31,330.0,20,Olympic,2022,33,M,O,...,F,11360(A)HS,transport/sell/furnish/etc marijuana,WESTERN,4TH,"['34.0672', '-118.3091']",34.0672,-118.3091,transport/sell/furnish/etc marijuana,2014
17694,1009080,4195723,2014-12-30,2110.0,11,Northeast,1159,46,M,H,...,F,11359HS,possession marijuana for sale,SYCAMORE,FIGUEROA,"['34.0994', '-118.2035']",34.0994,-118.2035,possession marijuana for sale,2014


In [72]:
for year in sorted(df_prisoes['Year'].unique()):
    display(grafico_prisoes_lojas(str(year)))

None

None

None

None

None

None

As prisões foram concentradas em uma região da cidade e claramente próximas a pontos de vendas, antes e depois da legalização da maconha. As lojas não mudaram de lugar e ao longo dos anos mais lojas surgiram. Se houve redução do volume de prisões, isso deve-se provavelmente à legalização da maconha.

E com relação às escolas? Vamos checar.

<h3>2.3) Prisões Associadas à Maconha e Que Ocorreram Próximo a Escolas </h3>

In [74]:
# Carregando os dados
df_escolas = pd.read_csv('dados/df_escolas.csv')

# Shape
df_escolas.shape

(947, 7)

In [75]:
# Dados
df_escolas.head()

,Unnamed: 0,School,Zip Code,Grades,Local District,Board District,complete_address
0,0,Frank Del Olmo Elementary,90004,K- 5,C,2-Monica Garcia,100 N New Hampshire Ave Los Angeles CA 90004
1,1,10th Street Elementary,90015,1- 5,C,2-Monica Garcia,1000 Grattan St Los Angeles CA 90015
2,2,Elementary Community Day School,91311,K- 6,XS,3-Scott M Schmerelson,10001 Jumilla Ave Chatsworth CA 91311
3,3,Cesar E Chavez Learning Academy - Arts/Theatre...,91340,9-12,NE,6-Kelly Gonez,1001 Arroyo Ave San Fernando CA 91340
4,4,San Pedro Senior High Gifted STEAM Magnet,90731,9-12,S,7-Dr Richard A Vladovic,1001 W 15th St San Pedro CA 90731


Temos um pequeno problema aqui. Precisamos da latitude e longitude para poder criar os gráficos, mas temos apenas o endereço completo. Vamos resolver isso com **Geolocalização**. Precisamos da latitude e longitude e temos apenas o endereço completo. A partir do endereço, buscaremos latitude e longitude.

Eu criei um dataset contendo toda a latitude e longitude das escolas.

In [81]:
# Carregando os dados
df_escolas = pd.read_csv('dados/df_escolas_parte2.csv')
# Dados
df_escolas.head()

,School,Zip Code,Grades,Local District,Board District,complete_address,latitude,longitude
0,Frank Del Olmo Elementary,90004,K- 5,C,2-Monica Garcia,100 N New Hampshire Ave Los Angeles CA 90004,34.072925,-118.292591
1,10th Street Elementary,90015,1- 5,C,2-Monica Garcia,1000 Grattan St Los Angeles CA 90015,34.048284,-118.273264
2,Elementary Community Day School,91311,K- 6,XS,3-Scott M Schmerelson,10001 Jumilla Ave Chatsworth CA 91311,34.251825,-118.563377
3,Cesar E Chavez Learning Academy - Arts/Theatre...,91340,9-12,NE,6-Kelly Gonez,1001 Arroyo Ave San Fernando CA 91340,34.288514,-118.421011
4,San Pedro Senior High Gifted STEAM Magnet,90731,9-12,S,7-Dr Richard A Vladovic,1001 W 15th St San Pedro CA 90731,33.730370,-118.299342
